In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
import json

In [3]:
with open('../temp_files/route_dict_journal.json', 'r') as json_file:
    route_dict = json.load(json_file)

In [4]:
route_dict['1']['link_idx_list']

[92, 81, 84, 112]

In [5]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

### create route_choice_probability_matrix $\bf{P} = \left[p_{ir}\right]$ (\# of OD pairs $\times$ \# of routes)

In [6]:
od_nodes_dict = zload('../temp_files/od_nodes_dict_journal.pkz')

In [7]:
od_nodes_dict

{0: {1},
 1: {2, 4, 5},
 2: {3},
 3: {6, 8},
 4: {7, 9},
 5: {10, 11},
 6: {12, 15},
 7: {13, 14},
 8: {16, 17, 18, 19},
 9: {20},
 10: {21},
 11: {22},
 12: {23, 24, 25, 26, 27, 28},
 13: {29},
 14: {30, 31, 32, 33, 34},
 15: {35, 71},
 16: {36},
 17: {37, 38, 39, 40, 41},
 18: {42},
 19: {43, 44, 45, 46, 47},
 20: {48, 74},
 21: {49, 73},
 22: {50, 51},
 23: {52},
 24: {53},
 25: {54, 68, 70},
 26: {55, 56},
 27: {57},
 28: {58},
 29: {59, 60, 72},
 30: {61},
 31: {62, 63, 64, 65, 66},
 32: {67},
 33: {69}}

In [8]:
od_pair_label_dict = {}

idx = 0
for origi in range(len(od_nodes_dict)):
    for desti in range(len(od_nodes_dict)):
        if origi != desti:
            od_pair_label_dict[idx] = [origi, desti]
            idx += 1
            
zdump(od_pair_label_dict, '../temp_files/od_pair_label_dict_journal.pkz')

In [9]:
od_pair_label_dict

{0: [0, 1],
 1: [0, 2],
 2: [0, 3],
 3: [0, 4],
 4: [0, 5],
 5: [0, 6],
 6: [0, 7],
 7: [0, 8],
 8: [0, 9],
 9: [0, 10],
 10: [0, 11],
 11: [0, 12],
 12: [0, 13],
 13: [0, 14],
 14: [0, 15],
 15: [0, 16],
 16: [0, 17],
 17: [0, 18],
 18: [0, 19],
 19: [0, 20],
 20: [0, 21],
 21: [0, 22],
 22: [0, 23],
 23: [0, 24],
 24: [0, 25],
 25: [0, 26],
 26: [0, 27],
 27: [0, 28],
 28: [0, 29],
 29: [0, 30],
 30: [0, 31],
 31: [0, 32],
 32: [0, 33],
 33: [1, 0],
 34: [1, 2],
 35: [1, 3],
 36: [1, 4],
 37: [1, 5],
 38: [1, 6],
 39: [1, 7],
 40: [1, 8],
 41: [1, 9],
 42: [1, 10],
 43: [1, 11],
 44: [1, 12],
 45: [1, 13],
 46: [1, 14],
 47: [1, 15],
 48: [1, 16],
 49: [1, 17],
 50: [1, 18],
 51: [1, 19],
 52: [1, 20],
 53: [1, 21],
 54: [1, 22],
 55: [1, 23],
 56: [1, 24],
 57: [1, 25],
 58: [1, 26],
 59: [1, 27],
 60: [1, 28],
 61: [1, 29],
 62: [1, 30],
 63: [1, 31],
 64: [1, 32],
 65: [1, 33],
 66: [2, 0],
 67: [2, 1],
 68: [2, 3],
 69: [2, 4],
 70: [2, 5],
 71: [2, 6],
 72: [2, 7],
 73: [2, 8],


In [10]:
# number of OD pairs
number_of_od_pairs = len(od_pair_label_dict)

In [11]:
od_pair_label_dict[0]

[0, 1]

In [12]:
od_pair_routes_dict = {}
for row in range(number_of_od_pairs):
    routes = []
    for column in range(len(route_dict)):
        if route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            routes.append(column)
            
    route_length_list = [route_dict[str(column)]['length_of_route'] for column in routes]
    
    refined_routes = []

    number_of_routes_per_OD_pair = 10
    if len(routes) <= number_of_routes_per_OD_pair:
        refined_routes = routes
    else:
        refined_idx_list = [np.argsort(route_length_list)[i] for i in range(number_of_routes_per_OD_pair)]
        refined_routes = [routes[idx] for idx in refined_idx_list]
        
    od_pair_routes_dict[row] = refined_routes

In [13]:
od_pair_routes_dict

{0: [3658, 3659, 3660, 5163, 5164, 5165, 5863, 5864, 5865],
 1: [4325, 4326, 4327],
 2: [6549, 6550, 6551, 8050, 8051, 8052],
 3: [8740, 8741, 8742, 14634, 14635, 14636],
 4: [1993, 1994, 1995, 12237, 12238, 12239],
 5: [5214, 5215, 5216, 5217, 5218, 5219],
 6: [5211, 5212, 5213, 5220, 5221, 5222],
 7: [5223, 5224, 5226, 5227, 5225, 5228, 5229, 5230, 5231, 5232],
 8: [5238, 5239, 5240],
 9: [5235, 5236, 5237],
 10: [8500, 8501, 8502],
 11: [8503, 8504, 8506, 8505, 8494, 8488, 8509, 8510, 8507, 8508],
 12: [8497, 8498, 8499],
 13: [12024, 12025, 12026, 8524, 8521, 8525, 8522, 8526, 12021, 12027],
 14: [6774, 6775, 6776, 12018, 12019, 12020],
 15: [12039, 12040, 12041],
 16: [10089, 12006, 10090, 10091, 12030, 12007, 12008, 12031, 12032, 12036],
 17: [15436, 15437, 15438],
 18: [14568, 14569, 14550, 14570, 14551, 14552, 15430, 15424, 448, 15431],
 19: [9376, 9377, 9378, 15418, 15419, 15420],
 20: [9379, 9380, 9381, 15421, 15422, 15423],
 21: [4163, 4164, 4165, 14464, 14465, 14466],
 22: 

In [14]:
# re-label routes
new_route_label_dict = {}
route_idx = 0
for od_pair_idx in range(len(od_pair_routes_dict)):
    for route in od_pair_routes_dict[od_pair_idx]:
        new_route_label_dict[route_idx] = route
        route_idx += 1

In [15]:
new_route_label_dict

{0: 3658,
 1: 3659,
 2: 3660,
 3: 5163,
 4: 5164,
 5: 5165,
 6: 5863,
 7: 5864,
 8: 5865,
 9: 4325,
 10: 4326,
 11: 4327,
 12: 6549,
 13: 6550,
 14: 6551,
 15: 8050,
 16: 8051,
 17: 8052,
 18: 8740,
 19: 8741,
 20: 8742,
 21: 14634,
 22: 14635,
 23: 14636,
 24: 1993,
 25: 1994,
 26: 1995,
 27: 12237,
 28: 12238,
 29: 12239,
 30: 5214,
 31: 5215,
 32: 5216,
 33: 5217,
 34: 5218,
 35: 5219,
 36: 5211,
 37: 5212,
 38: 5213,
 39: 5220,
 40: 5221,
 41: 5222,
 42: 5223,
 43: 5224,
 44: 5226,
 45: 5227,
 46: 5225,
 47: 5228,
 48: 5229,
 49: 5230,
 50: 5231,
 51: 5232,
 52: 5238,
 53: 5239,
 54: 5240,
 55: 5235,
 56: 5236,
 57: 5237,
 58: 8500,
 59: 8501,
 60: 8502,
 61: 8503,
 62: 8504,
 63: 8506,
 64: 8505,
 65: 8494,
 66: 8488,
 67: 8509,
 68: 8510,
 69: 8507,
 70: 8508,
 71: 8497,
 72: 8498,
 73: 8499,
 74: 12024,
 75: 12025,
 76: 12026,
 77: 8524,
 78: 8521,
 79: 8525,
 80: 8522,
 81: 8526,
 82: 12021,
 83: 12027,
 84: 6774,
 85: 6775,
 86: 6776,
 87: 12018,
 88: 12019,
 89: 12020,
 90: 1

In [16]:
route_dict

{u'13357': {u'desti': 31,
  u'length_of_route': 49.41366182901165,
  u'link_idx_list': [66, 78, 88, 96, 125, 128, 230, 244, 247],
  u'node_list': [19, 22, 23, 24, 33, 32, 60, 63, 65, 64],
  u'origi': 8},
 u'13356': {u'desti': 31,
  u'length_of_route': 48.4937433549711,
  u'link_idx_list': [66, 78, 90, 122, 232, 247],
  u'node_list': [19, 22, 23, 31, 60, 65, 64],
  u'origi': 8},
 u'11542': {u'desti': 1,
  u'length_of_route': 52.158106030616565,
  u'link_idx_list': [116, 91, 79, 59, 54, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 16, 17, 6, 3, 2],
  u'origi': 14},
 u'11543': {u'desti': 1,
  u'length_of_route': 52.69422945554072,
  u'link_idx_list': [116, 91, 79, 61, 25, 9, 7],
  u'node_list': [30, 31, 23, 22, 17, 6, 3, 2],
  u'origi': 14},
 u'11540': {u'desti': 25,
  u'length_of_route': 5.334022101781262,
  u'link_idx_list': [254],
  u'node_list': [69, 70],
  u'origi': 33},
 u'11541': {u'desti': 1,
  u'length_of_route': 46.93644430865088,
  u'link_idx_list': [71, 43, 38, 31, 21, 9, 7],
 

In [17]:
new_route_dict = {}
for key in new_route_label_dict.keys():
    new_route_dict[key] = route_dict[str(new_route_label_dict[key])]

In [18]:
new_route_dict

{0: {u'desti': 1,
  u'length_of_route': 20.081937967805782,
  u'link_idx_list': [0, 7],
  u'node_list': [1, 3, 2],
  u'origi': 0},
 1: {u'desti': 1,
  u'length_of_route': 31.769899478151604,
  u'link_idx_list': [2, 11, 7],
  u'node_list': [1, 7, 3, 2],
  u'origi': 0},
 2: {u'desti': 1,
  u'length_of_route': 37.241348544153276,
  u'link_idx_list': [4, 27, 11, 7],
  u'node_list': [1, 9, 7, 3, 2],
  u'origi': 0},
 3: {u'desti': 1,
  u'length_of_route': 27.203765398150747,
  u'link_idx_list': [0, 8, 13],
  u'node_list': [1, 3, 6, 4],
  u'origi': 0},
 4: {u'desti': 1,
  u'length_of_route': 33.067133130729246,
  u'link_idx_list': [0, 8, 20, 15],
  u'node_list': [1, 3, 6, 8, 4],
  u'origi': 0},
 5: {u'desti': 1,
  u'length_of_route': 35.47496951234316,
  u'link_idx_list': [4, 36, 23, 13],
  u'node_list': [1, 9, 13, 6, 4],
  u'origi': 0},
 6: {u'desti': 1,
  u'length_of_route': 44.161875204994864,
  u'link_idx_list': [0, 8, 20, 30, 19],
  u'node_list': [1, 3, 6, 8, 11, 5],
  u'origi': 0},
 7: 

In [19]:
with open('../temp_files/new_route_dict_journal.json', 'w') as json_file:
    json.dump(new_route_dict, json_file)

In [20]:
with open('../temp_files/new_route_dict_journal.json', 'r') as json_file:
    new_route_dict = json.load(json_file)

In [21]:
# number of routes
number_of_routes = len(new_route_dict)

In [22]:
number_of_routes

8432

In [23]:
# initialize the route_choice_probability_matrix P
P = np.zeros((number_of_od_pairs, number_of_routes))

np.shape(P)

(1122, 8432)

In [24]:
for row in range(number_of_od_pairs):
    for column in range(number_of_routes):
        if new_route_dict[str(column)]['origi'] == od_pair_label_dict[row][0] and \
        new_route_dict[str(column)]['desti'] == od_pair_label_dict[row][1]:
            P[row, column] = 1

In [25]:
# for column in range(r):
#     assert(sum(P[:, column]) <= 1)

In [26]:
# for row in range(i):
#     assert(sum(P[row, :]) >= 1)

### Use sparse matrix to save memory

In [27]:
P = csr_matrix(P)

In [28]:
zdump(P, '../temp_files/OD_pair_route_incidence_journal.pkz')

In [29]:
link_label_dict = zload('../temp_files/link_label_dict_journal.pkz')
link_label_dict_ = zload('../temp_files/link_label_dict_journal_.pkz')

In [30]:
len(link_label_dict)

258

In [31]:
# number of links
number_of_links = len(link_label_dict)

In [32]:
number_of_links

258

In [33]:
# initialize the link_route incidence matrix
A = np.zeros((number_of_links, number_of_routes))

# create link_route incidence matrix A
# route index
for i in range(number_of_routes):
    # link index
    for j in range(number_of_links):
        if j in new_route_dict[str(i)]['link_idx_list']:
            A[j, i] = 1

### Use sparse matrix to save memory

In [34]:
A = csc_matrix(A)        

zdump(A, '../temp_files/link_route_incidence_matrix_journal.pkz')

In [35]:
matrix_rank(A.todense())

257

In [36]:
print(np.size(A, 0))
print(np.size(A, 1))

258
8432
